In [1]:
import pandas as pd
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
data=pd.read_csv('/Users/vedantgupta/Documents/summer project/data-analysis-with-technical-indicators-2/nifty_sensex_close_data/data/M&M.NS_2011_to_2023.csv')
data

,Date,M&M.NS_open,M&M.NS_high,M&M.NS_low,M&M.NS_close
0,2011-01-03,357.862466,360.991939,353.111985,354.620422
1,2011-01-04,354.417744,356.624142,349.059393,351.783600
2,2011-01-05,351.783627,355.273321,344.556590,347.190735
3,2011-01-06,349.869920,350.320203,341.314542,347.280792
4,2011-01-07,344.106288,344.871775,329.629678,333.344513
...,...,...,...,...,...
3199,2023-12-22,1630.000000,1652.000000,1622.750000,1634.250000
3200,2023-12-26,1640.900024,1669.000000,1634.500000,1662.250000
3201,2023-12-27,1670.000000,1691.800049,1660.250000,1687.949951
3202,2023-12-28,1688.949951,1739.949951,1688.400024,1734.449951


In [3]:
df=data[['Date','M&M.NS_close']] 

In [4]:
df.dtypes

Date             object
M&M.NS_close    float64
dtype: object

In [5]:
#parameters
n = 12
m = 26
t = 9

In [6]:
df.loc[:,'EMAn'] = df['M&M.NS_close'].ewm(span=n, adjust=False).mean()

df.loc[:,'EMAm'] = df['M&M.NS_close'].ewm(span=m, adjust=False).mean()


/var/folders/90/5g2l2f2j191dmr85qjr8jrn40000gn/T/ipykernel_91031/1350891437.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'EMAn'] = df['M&M.NS_close'].ewm(span=n, adjust=False).mean()
/var/folders/90/5g2l2f2j191dmr85qjr8jrn40000gn/T/ipykernel_91031/1350891437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'EMAm'] = df['M&M.NS_close'].ewm(span=m, adjust=False).mean()


In [7]:
for i in range(n):
    df.loc[i,'EMAn'] = None
for i in range(m):
    df.loc[i,'EMAm'] = None
    

In [8]:
df['MACD'] = df['EMAn'] - df['EMAm']
df['Signal Line'] = df['MACD'].ewm(span=t, adjust=False).mean()
df

,Date,M&M.NS_close,EMAn,EMAm,MACD,Signal Line
0,2011-01-03,354.620422,NaN,NaN,NaN,NaN
1,2011-01-04,351.783600,NaN,NaN,NaN,NaN
2,2011-01-05,347.190735,NaN,NaN,NaN,NaN
3,2011-01-06,347.280792,NaN,NaN,NaN,NaN
4,2011-01-07,333.344513,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3199,2023-12-22,1634.250000,1660.519209,1636.174464,24.344746,32.384323
3200,2023-12-26,1662.250000,1660.785485,1638.105985,22.679500,30.443358
3201,2023-12-27,1687.949951,1664.964634,1641.798131,23.166503,28.987987
3202,2023-12-28,1734.449951,1675.654682,1648.661228,26.993454,28.589081


In [9]:
trade_sig=np.zeros([len(df),1],dtype=float)

for i in range(m+1,len(df)):
    if df.loc[i,'MACD'] > df.loc[i,'Signal Line'] and df.loc[i-1,'MACD'] < df.loc[i-1,'Signal Line']:
        trade_sig[i] = 1
    if df.loc[i,'MACD'] < df.loc[i,'Signal Line'] and df.loc[i-1,'MACD'] > df.loc[i-1,'Signal Line']:
        trade_sig[i] = -1

In [10]:
trade_sig=trade_sig.reshape(len(trade_sig),1)

In [11]:
trade_sig.shape

(3204, 1)

In [12]:
df_numpy=df.to_numpy()

In [13]:
final=np.column_stack((df_numpy,trade_sig))

In [14]:
final_df=pd.DataFrame(final,columns=['Date','Close','EMAn','EMAm','MACD','Signal Line','Trade Signal'])

In [15]:
final_df.drop(["EMAn","EMAm","MACD","Signal Line"],axis=1,inplace=True)

In [18]:
final_df['Close'] = final_df['Close'].astype(float) 
final_df['Trade Signal'] = final_df['Trade Signal'].astype(float)

In [19]:
final_df.dtypes

Date             object
Close           float64
Trade Signal    float64
dtype: object

In [20]:
final_df.to_csv('/Users/vedantgupta/Documents/summer project/data-analysis-with-technical-indicators-2/nifty_sensex_close_data/data/trading_signal_indi_mm.csv',index=False)